# Food Safety
## Data Analysis with Pandas

## Dept of Public Health - SFO Resturants Safety Scores

In this project, we will investigate restaurant food safety scores for restaurants in San Francisco. The scores and violation information have been [made available by the San Francisco Department of Public Health](https://data.sfgov.org/Health-and-Social-Services/Restaurant-Scores-LIVES-Standard/pyih-qa8i). The main goal for this assignment is to walk through the process of Data Cleaning and EDA. 

In [1]:
import numpy as np
import pandas as pd 

import bz2 # Used to read compressed data
import os # Used to interact with the file system

## Obtaining the Data

### File Systems and I/O



Basic command intro.

In [2]:
!ls 

Data             Project 1A.ipynb


In [3]:
!ls data

bus.csv.bz2          ins2vio.csv.bz2      vio.csv.bz2
ins.csv.bz2          sf_zipcodes (1).json


We are going to use the [`pathlib`](https://docs.python.org/3/library/pathlib.html#basic-use) module to represent our file system paths and perform operations which allow us to learn more about the contents of our data. Note what `pathlib.Path.cwd()` outputs in relation to the output of `!ls` above.

<br/><br/><br/>

---

<br/><br/><br/>

# 1: Examining the Files
Let's first focus on understanding the structure of the data; this involves answering analysis queries such as:

* How much data do we have?
* Is the data in a standard format or encoding?
* Is the data organized in records?
* What are the fields in each record?

Let's start by looking at the contents of `data/`. This is not just a single file but rather a directory of multiple compressed files. We could inspect our data by uncompressing each file but in this project we're going to do almost everything in Python for maximum portability.

You will want to use a few useful python functions.  To move through the local filesystem you can use the `Path` module in `pathlinb`.  For example, to list the current directory you can [Path.cwd](https://docs.python.org/3/library/pathlib.html#pathlib.Path.cwd).

In [4]:
from pathlib import Path

Path.cwd()

PosixPath('/Users/ketakidingre/Project 1A')

The function returns a `pathlib.Path` object representing the location of the file.  It can also be used to list contents of directories and many other things. 

You will also need to work with `bzip2` files and you will want to be able to read their contents using the [bz2](https://docs.python.org/3/library/bz2.html) python library.

In [5]:
with bz2.open("data/bus.csv.bz2", "r") as f:
    print("The first line:", "\n\t", f.readline())

The first line: 
	 b'"business id column","name","address","city","state","postal_code","latitude","longitude","phone_number"\n'


<br/><br/><br/>

---


Implemented the `list_files`, `get_file_size`, and `get_linecount_bz2` functions to return the list of files in the directory, the sizes (in bytes) of a file, and the number of lines in the file.  Note the last `get_linecount_bz2` should not produce any intermediate files in the filesystem and should avoid storing the entire file in memory (don't do `len(file.readlines())`).

In [6]:
def list_files(directory):
    """
    Return a list of pathlib.Path objects for the files in the directory.
    
    directory: a string describing the directory to list 
        for example 'data/'
    """
    ...
    p = Path(directory)
    return [x for x in p.iterdir() if x.is_file()]
    


def get_file_size(file_name):
    """
    Return file size for a given filename.
    """ 
    ...
    p = Path(file_name)
    return p.stat().st_size

def get_linecount_bz2(file_name):
    """
    Returns the number of lines in bz2 file.  
    """ 
    ...
    count = 0 
    with bz2.open(file_name, "rb") as f:
        for line in f:
            count=count+1
    return count



In [7]:
info = []
for f in list_files("data/"):
    name = str(f)
    if name[-3:] == "bz2": 
        size = get_file_size(f)
        linecount = get_linecount_bz2(f)
        info.append({"name": name, "size": size, "linecount": linecount})

file_info = pd.DataFrame(info).sort_values("size")
file_info

,name,size,linecount
2,data/vio.csv.bz2,1337,66
3,data/ins.csv.bz2,110843,26664
0,data/bus.csv.bz2,113522,6254
1,data/ins2vio.csv.bz2,146937,40211


<br/><br/><br/>

---

 function `head_bz2` to return a list of the first `nlines` lines of each file. 
Using your `head_bz2` function implement the following `print_head_bz2` function that uses `print()` to print the filename followed by the first `nlines` of each file and their line numbers in the following format.

```
data/bus.csv.bz2
0 :	 b'"business id column","name","address","city","state","postal_code","latitude","longitude","phone_number"\n'
1 :	 b'"1000","HEUNG YUEN RESTAURANT","3279 22nd St","San Francisco","CA","94110","37.755282","-122.420493","-9999"\n'
2 :	 b'"100010","ILLY CAFFE SF_PIER 39","PIER 39  K-106-B","San Francisco","CA","94133","-9999","-9999","+14154827284"\n'
3 :	 b'"100017","AMICI\'S EAST COAST PIZZERIA","475 06th St","San Francisco","CA","94103","-9999","-9999","+14155279839"\n'
4 :	 b'"100026","LOCAL CATERING","1566 CARROLL AVE","San Francisco","CA","94124","-9999","-9999","+14155860315"\n'
```


<!--
BEGIN QUESTION
name: q1b
points: 2
-->

In [8]:
def head_bz2(file, nlines=5):
    """
    Return a list of the first nlines lines of filename
    """
    ...
    line_list = []
    with bz2.open(file,'r')as f:
        for line in np.arange(nlines):
            this_line = f.readline()
            line_list.append(this_line)
        return line_list

def print_head_bz2(file, nlines=5):
    """
    Print a list of the first nlines lines of filename
    """
    ...
    print(file)
    line_num = 0
    for x in head_bz2(file,nlines):
        line_num +=1
        print(str(line_num) + ":" + " " + str(x))
        print(x)
    

In [9]:
for file in list_files("data/"):
    if str(file)[-3:] == "bz2":  
        print_head_bz2(file)
        print()

data/bus.csv.bz2
1: b'"business id column","name","address","city","state","postal_code","latitude","longitude","phone_number"\n'
b'"business id column","name","address","city","state","postal_code","latitude","longitude","phone_number"\n'
2: b'"1000","HEUNG YUEN RESTAURANT","3279 22nd St","San Francisco","CA","94110","37.755282","-122.420493","-9999"\n'
b'"1000","HEUNG YUEN RESTAURANT","3279 22nd St","San Francisco","CA","94110","37.755282","-122.420493","-9999"\n'
3: b'"100010","ILLY CAFFE SF_PIER 39","PIER 39  K-106-B","San Francisco","CA","94133","-9999","-9999","+14154827284"\n'
b'"100010","ILLY CAFFE SF_PIER 39","PIER 39  K-106-B","San Francisco","CA","94133","-9999","-9999","+14154827284"\n'
4: b'"100017","AMICI\'S EAST COAST PIZZERIA","475 06th St","San Francisco","CA","94103","-9999","-9999","+14155279839"\n'
b'"100017","AMICI\'S EAST COAST PIZZERIA","475 06th St","San Francisco","CA","94103","-9999","-9999","+14155279839"\n'
5: b'"100026","LOCAL CATERING","1566 CARROLL AVE","

<br/><br/><br/>

---

Thinking about the files

Analyze the following queries by filling in the correct boolean values in the following variables:

1. The bus.csv.bz2 file appears to be tab delimited.
1. The values all appear to be quoted.

In [10]:
# True or False: The bus.csv.bz2 file appears to be tab delimited.
q1c_1 = ...
False

# True or False: The values all appear to be quoted.
q1c_2 = ...
True

True

<br/><br/><br/>

---

Reading in the Files

Based on the above information, let's attempt to load `bus.csv`, `ins2vio.csv`, `vio.csv` and `ins.csv` into Pandas dataframes with the following names: `bus`, `ins2vio`, `vio`, `ins` respectively.

In [11]:
# path to directory containing data
bus = pd.read_csv('data/bus.csv.bz2')
ins = pd.read_csv('data/ins.csv.bz2')
ins2vio = pd.read_csv('data/ins2vio.csv.bz2')
vio = pd.read_csv('data/vio.csv.bz2')

Now that you've read in the files, you can try some `pd.DataFrame` methods ([docs](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.html)).
You can use the `DataFrame.head` method to show the top few lines of the `bus`, `ins`, `ins2vio` and `vio` dataframes.

In [12]:
bus.head(25)

,business id column,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675
5,100036,Hula Truck (#2),2 Marina Blvd,San Francisco,CA,94123,-9999.000000,-9999.000000,-9999
6,100039,GENKI CREPES & MINI MART,330 CLEMENT ST,San Francisco,CA,94118,-9999.000000,-9999.000000,14155376414
7,100041,UNCLE LEE CAFE,3608 BALBOA ST,San Francisco,CA,94121,-9999.000000,-9999.000000,-9999
8,100055,Twirl and Dip,335 Martin Luther King Jr. Dr,San Francisco,CA,94118,-9999.000000,-9999.000000,14155300260
9,100058,SF PITA HUB,475 06TH ST,San Francisco,CA,94103,-9999.000000,-9999.000000,14155642006


In [13]:
ins.head(10)

,iid,date,score,type
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup
5,100017_20190912,09/12/2019 12:00:00 AM,-1,Reinspection/Followup
6,100026_20190418,04/18/2019 12:00:00 AM,-1,New Ownership
7,100030_20190612,06/12/2019 12:00:00 AM,-1,New Ownership
8,100030_20190826,08/26/2019 12:00:00 AM,-1,New Ownership
9,100036_20190325,03/25/2019 12:00:00 AM,-1,Structural Inspection


In [14]:
ins2vio.head()

,iid,vid
0,97975_20190725,103124
1,85986_20161011,103114
2,95754_20190327,103124
3,77005_20170429,103120
4,4794_20181030,103138


In [15]:
vio.head()

,description,risk_category,vid
0,Consumer advisory not provided for raw or unde...,Moderate Risk,103128
1,Contaminated or adulterated food,High Risk,103108
2,Discharge from employee nose mouth or eye,Moderate Risk,103117
3,Employee eating or smoking,Moderate Risk,103118
4,Food in poor condition,Moderate Risk,103123


<br/><br/><br/>

---

##  Identifying Issues with the Data

Use the `head` command on your four files again. This time, describe at least one potential problem with the data you see.

*Write your answer here, replacing this text.*

In [316]:
# Show work here.
bus.head()

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number,postal5
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999,None
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284,None
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839,None
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315,None
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675,None


In [317]:

ins.head()

,iid,date,score,type,bid,timestamp,year,boolean_c
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,100010,2019-03-29,2019,True
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010,2019-04-03,2019,False
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,100017,2019-04-17,2019,True
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017,2019-08-16,2019,False
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,100017,2019-08-26,2019,True


In [319]:

ins2vio.head()

,iid,vid
0,97975_20190725,103124
1,85986_20161011,103114
2,95754_20190327,103124
3,77005_20170429,103120
4,4794_20181030,103138


In [318]:

vio.head()

,description,risk_category,vid
0,Consumer advisory not provided for raw or unde...,Moderate Risk,103128
1,Contaminated or adulterated food,High Risk,103108
2,Discharge from employee nose mouth or eye,Moderate Risk,103117
3,Employee eating or smoking,Moderate Risk,103118
4,Food in poor condition,Moderate Risk,103123


<br/><br/><br/>

---

<br/><br/><br/>

# 2: Examining the Business Data File

From its name alone, we expect the `bus.csv` file to contain information about the restaurants. Let's investigate the granularity of this dataset.

In [17]:
bus.head()

,business id column,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675


<br/><br/><br/>

---



The `bus` dataframe contains a column called `business id column` which probably corresponds to a unique business id.  However, let's first rename that column to `bid`.  Modify the `bus` dataframe by renaming that column to `bid`.

**Note**: In practice we might want to do this renaming when the table is loaded but for grading purposes we will do it here.

In [18]:
bus = bus.rename(columns={"business id column":"bid"})
bus

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number
0,1000,HEUNG YUEN RESTAURANT,3279 22nd St,San Francisco,CA,94110,37.755282,-122.420493,-9999
1,100010,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B,San Francisco,CA,94133,-9999.000000,-9999.000000,14154827284
2,100017,AMICI'S EAST COAST PIZZERIA,475 06th St,San Francisco,CA,94103,-9999.000000,-9999.000000,14155279839
3,100026,LOCAL CATERING,1566 CARROLL AVE,San Francisco,CA,94124,-9999.000000,-9999.000000,14155860315
4,100030,OUI OUI! MACARON,2200 JERROLD AVE STE C,San Francisco,CA,94124,-9999.000000,-9999.000000,14159702675
...,...,...,...,...,...,...,...,...,...
6248,99948,SUSIECAKES BAKERY,3509 CALIFORNIA ST,San Francisco,CA,94118,-9999.000000,-9999.000000,14150452253
6249,99988,HINODEYA SOMA,303 02nd ST STE 102,San Francisco,CA,94107,-9999.000000,-9999.000000,-9999
6250,99991,TON TON,422 GEARY ST,San Francisco,CA,94102,-9999.000000,-9999.000000,14155531280
6251,99992,URBAN EXPRESS KITCHENS LLC,475 06th ST,San Francisco,CA,94103,-9999.000000,-9999.000000,14150368085


<br/><br/><br/>

---



Examining the entries in `bus`, is the `bid` unique for each record (i.e. each row of data)? Your code should compute the answer, i.e. don't just hard code `True` or `False`.

Hint: use `value_counts()` or `unique()` to determine if the `bid` series has any duplicates.


In [19]:
is_bid_unique = len(pd.value_counts(bus["bid"]))==len(pd.unique(bus["bid"]))
is_bid_unique

True

<br/><br/><br/>

---




In the two cells below create two **series** 

1. where the index is the `name` of the business and the value is the number of records with that `name`
2. where the index is the `address` of the business and the value is the number of records with that `address`

Order both series in descending order by count. You may need to use `groupby()`, `size()`, `sort_values()`, or `value_counts()`. 

**Step 1**

In [20]:
name_counts = bus.value_counts("name")
name_counts.head()

name
Peet's Coffee & Tea    20
Starbucks Coffee       13
McDonald's             10
Jamba Juice            10
Proper Food             9
dtype: int64

**Step 2**

<!--
BEGIN QUESTION
name: q2cii
points: 1
-->

In [21]:
address_counts = bus.value_counts("address")
address_counts.head(10)


address
Off The Grid              39
428 11th St               34
3251 20th Ave             17
2948 Folsom St            17
Pier 41                   16
103 Horne Ave             14
24 Willie Mays Plaza      13
Off the Grid              11
2948 Folsom St.           10
1 United Nations Plaza    10
dtype: int64

<br/><br/><br/>

---



Based on the above calculations answer each of the following queries by filling the value in the variable.

1. What does each record represent?  
1. What is the minimal primary key?

In [320]:
# What does each record represent?  Valid answers are:
#    "One location of a restaurant."
#    "A chain of restaurants."
#    "A city block."
q2d_part1 = "One location of a restaurant."

# What is the minimal primary key? Valid answers are:
#    "bid"
#    "bid, name"
#    "bid, name, address"
q2d_part2 = "bid"

<br/><br/><br/>

---

<br/><br/><br/>

# 3: Cleaning the Business Data Postal Codes

The business data contains postal code information that we can use to aggregate the ratings over regions of the city.  Let's examine and clean the postal code field.  The postal code (sometimes also called a ZIP code) partitions the city into regions:

<img src="https://www.usmapguide.com/wp-content/uploads/2019/03/printable-san-francisco-zip-code-map.jpg" alt="ZIP Code Map" style="width: 600px">

<br/><br/><br/>

---




How many restaurants are in each ZIP code? 

In the cell below, create a **series** where the index is the postal code and the value is the number of records with that postal code in descending order of count. You may need to use `groupby()`, `size()`, or `value_counts()`. Do you notice any odd/invalid zip codes?

<!--
BEGIN QUESTION
name: q3a
points: 1
-->

In [23]:
zip_counts = bus.value_counts('postal_code')
print(zip_counts.to_string())

postal_code
94103         562
94110         555
94102         456
94107         408
94133         398
94109         382
94111         259
94122         255
94105         249
94118         231
94115         230
94108         229
94124         218
94114         200
-9999         194
94112         192
94117         189
94123         177
94121         157
94104         142
94132         132
94116          97
94158          90
94134          82
94127          67
94131          49
94130           8
94143           5
94101           2
CA              2
94188           2
94301           2
94013           2
94518           1
95133           1
95132           1
94602           1
94544           1
95122           1
94621           1
95105           1
95109           1
95112           1
95117           1
94901           1
94105-2907      1
64110           1
94120           1
941102019       1
941033148       1
94102-5917      1
941             1
94117-3504      1
94080           1
00000           

<br/><br/><br/>

--- 


Analyze below queries about the `postal_code` column in the `bus` dataframe.

1. The ZIP code column is which of the following type of data:
    1. Quantitative Continuous
    1. Quantitative Discrete
    1. Qualitative Ordinal
    1. Qualitative Nominal    
1. What Python data type is used to represent a ZIP code?

*Note*: ZIP codes and postal codes are the same thing.

In [24]:
# The ZIP code column is which of the following type of data:
#   "Quantitative Continuous" 
#   "Quantitative Discrete"
#   "Qualitative Ordinal"
#   "Qualitative Nominal"
q3b_part1 = "Qualitative Nominal"
# What Python data type is used to represent a ZIP code? 
#    "str"
#    "int"
#    "bool"
#    "float"
q3b_part2 = "str"

<br/><br/><br/>

--- 



Previously we noticed a large number of potentially invalid ZIP codes (e.g., "CA").  These are likely due to data entry errors.  To get a better understanding of the potential errors in the zip codes we will:

1. Import a list of valid San Francisco ZIP codes by using `pd.read_json` to load the file `data/sf_zipcodes.json` and extract a **series** of type `str` containing the valid ZIP codes.  *Hint: set `dtype` when invoking `read_json`.*
1. Construct a DataFrame containing only the businesses which DO NOT have valid ZIP codes.  You will probably want to use the `Series.isin` function. 


**Step 1**

<!--
BEGIN QUESTION
name: q3ci
points: 1
-->

In [35]:
valid_zips = pd.read_json("data/sf_zipcodes.json",dtype=str)['zip_codes']
valid_zips.head()

0    94102
1    94103
2    94104
3    94105
4    94107
Name: zip_codes, dtype: object

**Step 2**

<!--
BEGIN QUESTION
name: q3cii
points: 1
-->

In [39]:
invalid_zip_bus = bus[bus['postal_code'].isin(valid_zips)==False]
invalid_zip_bus.head(20)

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number
22,100126,Lamas Peruvian Food Truck,Private Location,San Francisco,CA,-9999,-9999.000000,-9999.000000,-9999
68,100417,"COMPASS ONE, LLC",1 MARKET ST. FL,San Francisco,CA,94105-1420,-9999.000000,-9999.000000,14154324000
96,100660,TEAPENTER,1518 IRVING ST,San Francisco,CA,94122-1909,-9999.000000,-9999.000000,14155868318
109,100781,LE CAFE DU SOLEIL,200 FILLMORE ST,San Francisco,CA,94117-3504,-9999.000000,-9999.000000,14155614215
144,101084,Deli North 200,1 Warriors Way Level 300 North East,San Francisco,CA,94518,-9999.000000,-9999.000000,-9999
156,101129,Vendor Room 200,1 Warriors Way Level 300 South West,San Francisco,CA,-9999,-9999.000000,-9999.000000,-9999
177,101192,Cochinita #2,2 Marina Blvd Fort Mason,San Francisco,CA,-9999,-9999.000000,-9999.000000,14150429222
276,102014,"DROPBOX (Section 3, Floor 7)",1800 Owens St,San Francisco,CA,-9999,-9999.000000,-9999.000000,-9999
295,102245,Vessell CA Operations (#4),2351 Mission St,San Francisco,CA,-9999,-9999.000000,-9999.000000,-9999
298,10227,The Napper Tandy,3200 24th St,San Francisco,CA,-9999,37.752581,-122.416482,-9999


<br/><br/><br/>

--- 



In the previous analysis, many of the businesses had a common invalid postal code that was likely used to code a MISSING postal code.  Do they all share a potentially "interesting address"?

In the following cell, construct a **series** that counts the number of businesses at each `address` that have this single likely MISSING postal code value.  Order the series in descending order by count. 

After examining the output.  If we were to drop businesses with MISSING postal code values would a particular class of business be affected?  If you are unsure try to search the web for the most common addresses.

In [45]:
missing = bus['postal_code']=='-9999'
missing_df = bus[missing]
missing_df

,bid,name,address,city,state,postal_code,latitude,longitude,phone_number
22,100126,Lamas Peruvian Food Truck,Private Location,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
156,101129,Vendor Room 200,1 Warriors Way Level 300 South West,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
177,101192,Cochinita #2,2 Marina Blvd Fort Mason,San Francisco,CA,-9999,-9999.0,-9999.0,14150429222
276,102014,"DROPBOX (Section 3, Floor 7)",1800 Owens St,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
295,102245,Vessell CA Operations (#4),2351 Mission St,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
...,...,...,...,...,...,...,...,...,...
5820,96774,DANIEL SCOTT RIPLEY CATERER,425 Barneveld Ave,San Francisco,CA,-9999,-9999.0,-9999.0,14155212714
5843,96938,POKE REESE,3251 Pierce,San Francisco,CA,-9999,-9999.0,-9999.0,14150463411
5940,97700,THE ITALIAN HOMEMADE COMPANY,1 franklin Ct,San Francisco,CA,-9999,-9999.0,-9999.0,-9999
6078,98788,333 Truck,Off The Grid,San Francisco,CA,-9999,-9999.0,-9999.0,-9999


In [47]:
missing_zip_address_count = pd.value_counts(missing_df['address'])
missing_zip_address_count

Off The Grid                  39
Off the Grid                  10
OTG                            4
Approved Locations             3
Approved Private Locations     3
                              ..
Macys - Geary Entrance         1
1001 Potrero Ave               1
550 A Gene Friend Way          1
550 Gene Friends Way           1
1 franklin Ct                  1
Name: address, Length: 135, dtype: int64



**True or False**:  *If we were to drop businesses with MISSING postal code values a particular class of business will be affected.*

In [48]:
# True or False: 
#  If we were to drop businesses with MISSING postal code values 
#   a particular class of business be affected.
q3d_true_or_false = True


<br/><br/><br/>

--- 



Examine the `invalid_zip_bus` dataframe we computed above and look at the businesses that DO NOT have the special MISSING ZIP code value.  Some of the invalid postal codes are just the full 9 digit code rather than the first 5 digits.  Create a new column named `postal5` in the original `bus` dataframe which contains only the first 5 digits of the `postal_code` column.   Finally, for any of the `postal5` ZIP code entries that were not a valid San Fransisco ZIP Code (according to `valid_zips`) set the entry to `None`. 

In [ ]:
bus['postal5'] = bus.apply(lambda x: x['postal_code'],axis=1)

In [75]:
bus['postal5'] = bus.apply(lambda x: x['postal_code'] if x['postal_code'] in (valid_zips) else x['postal_code'][:5] if len(x['postal_code']) > 5 else None,axis=1)


# Checking the corrected postal5 column
bus.loc[invalid_zip_bus.index, ['bid', 'name', 'postal_code', 'postal5']]

,bid,name,postal_code,postal5
22,100126,Lamas Peruvian Food Truck,-9999,None
68,100417,"COMPASS ONE, LLC",94105-1420,94105
96,100660,TEAPENTER,94122-1909,94122
109,100781,LE CAFE DU SOLEIL,94117-3504,94117
144,101084,Deli North 200,94518,None
...,...,...,...,...
6173,99369,HOTEL BIRON,94102-5917,94102
6174,99376,Mashallah Halal Food truck Ind,-9999,None
6199,99536,FAITH SANDWICH #2,94105-2907,94105
6204,99681,Twister,95112,None


<br/><br/><br/>

---

<br/><br/><br/>

# 4: Investigate the Inspection Data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns named 
`iid`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

Let's start by looking again at the first 5 rows of `ins` to see what we're working with.

In [76]:
ins.head(5)

,iid,date,score,type
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup


<br/><br/><br/>

---


The column `iid` probably corresponds to an inspection id.  Is it a primary key?  Write an expression (line of code) that evaluates to 'True' or 'False' based on whether all the values are unique.


In [115]:
is_ins_iid_a_primary_key = len(ins.iid.unique())==len(ins.iid)
is_ins_iid_a_primary_key

True

<br/><br/><br/>

---



The column `iid` appears to be the composition of two numbers and the first number looks like a business id.  

**Part 1.**: Create a new column called `bid` in the `ins` dataframe containing just the business id.  You will want to use `ins['iid'].str` operations to do this.  Also be sure to convert the type of this column to `int`

**Part 2.**: Then compute how many values in this new column are also valid business ids (appear in the `bus['bid']` column). This is verifying a foreign key relationship. Consider using the `pd.Series.isin` function.

**Part 3.**: Answer True or False, `ins['bid']` is a foreign key reference to `bus['bid']`.


**Part 1**

<!--
BEGIN QUESTION
name: q4bi
points: 1
-->

In [105]:
#ins['bid'] = ins['iid'].str.extract('')
ins['bid'] = ins ['iid'].str.split(pat='_', expand= True)[0]
ins['bid'].astype('int64')

0        100010
1        100010
2        100017
3        100017
4        100017
          ...  
26658       999
26659       999
26660       999
26661        99
26662        99
Name: bid, Length: 26663, dtype: int64

**Part 2**

<!--
BEGIN QUESTION
name: q4bii
points: 1
-->

In [140]:
invalid_bid_count = ins[ins['bid'].astype('int64').isin(bus['bid'])]
invalid_bid_count

,iid,date,score,type,bid
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,100010
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,100017
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,100017
...,...,...,...,...,...
26658,999_20180924,09/24/2018 12:00:00 AM,-1,Routine - Scheduled,999
26659,999_20181102,11/02/2018 12:00:00 AM,-1,Reinspection/Followup,999
26660,999_20190909,09/09/2019 12:00:00 AM,80,Routine - Unscheduled,999
26661,99_20171207,12/07/2017 12:00:00 AM,82,Routine - Unscheduled,99


**Part 3**

<!--
BEGIN QUESTION
name: q4biii
points: 1
-->

In [141]:
# True or False: The column ins['bid'] is a foreign key 
#   referencing the bus['bid'] primary key.

q4b_is_foreign_key = len(ins["bid"])== len(invalid_bid_count)
q4b_is_foreign_key

True

<br/><br/><br/>

---

What if we are interested in a time component of the inspection data?  We need to examine the date column of each inspection. 

**Part 1:** What is the type of the individual `ins['date']` entries. You may want to grab the very first entry and use the `type` function in python. 

**Part 2:** Use `pd.to_datetime` to create a new `ins['timestamp']` column containing of `pd.Timestamp` objects.  These will allow us to do more date manipulation.

**Part 3:** What are the earliest and latest dates in our inspection data?  *Hint: you can use `min` and `max` on dates of the correct type.*

**Part 4:** We probably want to examine the inspections by year. Create an additional `ins['year']` column containing just the year of the inspection.  Consider using `pd.Series.dt.year` to do this.

**No python `for` loops or list comprehensions required!**

**Part 1**

<!--
BEGIN QUESTION
name: q4ci
points: 1
-->

In [154]:
ins_date_type = type(ins['date'][0])
ins_date_type

str

**Part 2**

<!--
BEGIN QUESTION
name: q4cii
points: 1
-->

In [157]:
ins['timestamp'] = pd.to_datetime(ins["date"])
ins

,iid,date,score,type,bid,timestamp
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,100010,2019-03-29
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010,2019-04-03
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,100017,2019-04-17
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017,2019-08-16
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,100017,2019-08-26
...,...,...,...,...,...,...
26658,999_20180924,09/24/2018 12:00:00 AM,-1,Routine - Scheduled,999,2018-09-24
26659,999_20181102,11/02/2018 12:00:00 AM,-1,Reinspection/Followup,999,2018-11-02
26660,999_20190909,09/09/2019 12:00:00 AM,80,Routine - Unscheduled,999,2019-09-09
26661,99_20171207,12/07/2017 12:00:00 AM,82,Routine - Unscheduled,99,2017-12-07


**Part 3**

In [160]:
earliest_date = min(ins['timestamp'])
latest_date = max(ins['timestamp'])

print("Earliest Date:", earliest_date)
print("Latest Date:", latest_date)

Earliest Date: 2016-10-04 00:00:00
Latest Date: 2019-11-28 00:00:00


**Part 4** 

In [163]:
ins['year'] = ins['timestamp'].dt.year

In [164]:
ins.head()

,iid,date,score,type,bid,timestamp,year
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,100010,2019-03-29,2019
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010,2019-04-03,2019
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,100017,2019-04-17,2019
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017,2019-08-16,2019
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,100017,2019-08-26,2019


<br/><br/><br/>

---



What is the relationship between the type of inspection over the 2016 to 2019 timeframe? 

**Part 1**

Constructed the following table by
1. Using the `pivot_table` containing the number (`size`) of inspections for the given `type` and `year`.
1. Adding an extra `Total` column to the result using `sum`
1. Sort the results in descending order by the `Total`.



**Part 2**

Based on the above analysis, which year appears to have had a lot of businesses in new buildings?

**No python `for` loops or list comprehensions required!**

**Part 1** 

In [305]:
ins_pivot = pd.pivot_table(ins, values='iid', index=['type'],
                    columns=['year'], aggfunc= 'count',margins = True, margins_name='Total')
ins_pivot_sorted = ins_pivot.sort_values(by=('Total'), ascending=False)

ins_pivot_sorted

year,2016,2017,2018,2019,Total
type,,,,,
Total,1778.0,7817.0,8218.0,8850.0,26663
Routine - Unscheduled,966.0,4057.0,4373.0,4681.0,14077
Reinspection/Followup,445.0,1767.0,1935.0,2292.0,6439
New Ownership,99.0,506.0,528.0,459.0,1592
Complaint,91.0,418.0,512.0,437.0,1458
New Construction,102.0,485.0,218.0,189.0,994
Non-inspection site visit,51.0,276.0,253.0,231.0,811
New Ownership - Followup,NaN,45.0,219.0,235.0,499
Structural Inspection,1.0,153.0,50.0,190.0,394


**Part 2** 

In [313]:
ins_groups = ins_pivot.query('type == "New Construction"')
year_of_new_construction =  ins_groups.sort_values(by=('Total')) #ins_pivot.groupby('year').head(1).reset_index(drop=True) 
year_of_new_construction

year,2016,2017,2018,2019,Total
type,,,,,
New Construction,102.0,485.0,218.0,189.0,994


<br/><br/><br/>

---



Let's examine the inspection scores `ins['score']`

In [194]:
ins['score'].value_counts().head()

-1      12632
 100     1993
 96      1681
 92      1260
 94      1250
Name: score, dtype: int64

There are a large number of inspections with the `'score'` of `-1`.   These are probably missing values.  Let's see what type of inspections have scores and which do not.  Create the following dataframe using steps similar to the previous analysis.

You should observe that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections.


<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>Missing Score</th>      <th>False</th>      <th>True</th>      <th>Total</th>    </tr>    <tr>      <th>type</th>      <th></th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Routine - Unscheduled</th>      <td>14031</td>      <td>46</td>      <td>14077</td>    </tr>    <tr>      <th>Reinspection/Followup</th>      <td>0</td>      <td>6439</td>      <td>6439</td>    </tr>    <tr>      <th>New Ownership</th>      <td>0</td>      <td>1592</td>      <td>1592</td>    </tr>    <tr>      <th>Complaint</th>      <td>0</td>      <td>1458</td>      <td>1458</td>    </tr>    <tr>      <th>New Construction</th>      <td>0</td>      <td>994</td>      <td>994</td>    </tr>    <tr>      <th>Non-inspection site visit</th>      <td>0</td>      <td>811</td>      <td>811</td>    </tr>    <tr>      <th>New Ownership - Followup</th>      <td>0</td>      <td>499</td>      <td>499</td>    </tr>    <tr>      <th>Structural Inspection</th>      <td>0</td>      <td>394</td>      <td>394</td>    </tr>    <tr>      <th>Complaint Reinspection/Followup</th>      <td>0</td>      <td>227</td>      <td>227</td>    </tr>    <tr>      <th>Foodborne Illness Investigation</th>      <td>0</td>      <td>115</td>      <td>115</td>    </tr>    <tr>      <th>Routine - Scheduled</th>      <td>0</td>      <td>46</td>      <td>46</td>    </tr>    <tr>      <th>Administrative or Document Review</th>      <td>0</td>      <td>4</td>      <td>4</td>    </tr>    <tr>      <th>Multi-agency Investigation</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    <tr>      <th>Special Event</th>      <td>0</td>      <td>3</td>      <td>3</td>    </tr>    <tr>      <th>Community Health Assessment</th>      <td>0</td>      <td>1</td>      <td>1</td>    </tr>  </tbody></table>



In [209]:
ins['boolean_c'] = ins['score'] == -1
ins_missing_score_pivot = pd.pivot_table(ins, index='type', columns = 'boolean_c', values='score',aggfunc='count',
                                         dropna=True, observed=True,fill_value=0)
ins_missing_score_pivot['total'] = ins_missing_score_pivot.sum(axis=1)
ins_missing_score_pivot = ins_missing_score_pivot.sort_values('total', axis=0, ascending=False)
ins_missing_score_pivot

boolean_c,False,True,total
type,,,
Routine - Unscheduled,14031,46,14077
Reinspection/Followup,0,6439,6439
New Ownership,0,1592,1592
Complaint,0,1458,1458
New Construction,0,994,994
Non-inspection site visit,0,811,811
New Ownership - Followup,0,499,499
Structural Inspection,0,394,394
Complaint Reinspection/Followup,0,227,227


Notice that inspection scores appear only to be assigned to `Routine - Unscheduled` inspections. It is reasonable that for inspection types such as `New Ownership` and `Complaint` to have no associated inspection scores, but we might be curious why there are no inspection scores for the `Reinspection/Followup` inspection type.

<br/><br/><br/>

---

<br/><br/><br/>

# 5: Joining Data Across Tables

In this analysis we will start to connect data across mulitple tables.  We will be using the `merge` function.

<br/><br/><br/>

--- 



Let's figure out which restaurants had the lowest scores. Let's start by creating a new dataframe called `ins_named`. It should be exactly the same as `ins`, except that it should have the name and address of every business, as determined by the `bus` dataframe. 

*Hint*: Use the merge method to join the `ins` dataframe with the appropriate portion of the `bus` dataframe. See the official [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) on how to use `merge`.


*Note*: For quick reference, a pandas 'left' join keeps the keys from the left frame, so if ins is the left frame, all the keys from ins are kept and if a set of these keys don't have matches in the other frame, the columns from the other frame for these "unmatched" key rows contains NaNs.

In [210]:
bus['bid'] = bus['bid'].astype('str')
ins_named = ins.merge(bus[['name','address','bid']], how='left', on= 'bid')
ins_named.head()

,iid,date,score,type,bid,timestamp,year,boolean_c,name,address
0,100010_20190329,03/29/2019 12:00:00 AM,-1,New Construction,100010,2019-03-29,2019,True,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B
1,100010_20190403,04/03/2019 12:00:00 AM,100,Routine - Unscheduled,100010,2019-04-03,2019,False,ILLY CAFFE SF_PIER 39,PIER 39 K-106-B
2,100017_20190417,04/17/2019 12:00:00 AM,-1,New Ownership,100017,2019-04-17,2019,True,AMICI'S EAST COAST PIZZERIA,475 06th St
3,100017_20190816,08/16/2019 12:00:00 AM,91,Routine - Unscheduled,100017,2019-08-16,2019,False,AMICI'S EAST COAST PIZZERIA,475 06th St
4,100017_20190826,08/26/2019 12:00:00 AM,-1,Reinspection/Followup,100017,2019-08-26,2019,True,AMICI'S EAST COAST PIZZERIA,475 06th St


<br/><br/><br/>

--- 



Let's look at the 20 businesses with the lowest **median** score.  Order your results by the median score followed by the business id to break ties. The resulting table should look like:


*Hint: You may find the `as_index` argument important*

In [219]:
twenty = ins_named[ins_named['score'] >= 0]
twenty1 = twenty.groupby(by = "bid", as_index=False)
twenty2 = twenty1.agg({'name': 'first','score' :'median'})
twenty2sorted = twenty2.sort_values('score')
twenty2sorted.head(20)

,bid,name,score
3933,84590,Chaat Corner,54.0
4641,90622,Taqueria Lolita,57.0
5087,94351,VBowls LLC,58.0
2628,69282,New Jumbo Seafood Restaurant,60.5
188,1154,SUNFLOWER RESTAURANT,63.5
2643,69397,Minna SF Group LLC,64.0
5004,93502,Smoky Man,64.0
4962,93150,Chez Beesen,64.0
1389,39776,Duc Loi Supermarket,64.0
3325,78328,Golden Wok,64.0


<br/><br/><br/>

--- 


Let's now examine the descriptions of violations for inspections with `score > 0` and `score < 65`.  Construct a **Series** indexed by the `description` of the violation from the `vio` table with the value being the number of times that violation occured for inspections with the above score range.  Sort the results in descending order of the count.

The first few entries should look like:



In [224]:
low_score_violations = ins.merge(ins2vio, how='left', on ='iid')
mergejoinvio = low_score_violations.merge(vio, how='left', on = 'vid')
filterscorelt65 =mergejoinvio[mergejoinvio['score'] < 65]
filterscoregt0 = filterscorelt65[filterscorelt65['score'] > 0]
low_score_violations = filterscoregt0.groupby('description').size().sort_values(ascending=False)
low_score_violations.head(20)

description
Unclean or unsanitary food contact surfaces                                  43
High risk food holding temperature                                           42
Unclean or degraded floors walls or ceilings                                 40
Unapproved or unmaintained equipment or utensils                             39
Foods not protected from contamination                                       37
High risk vermin infestation                                                 37
Inadequate food safety knowledge or lack of certified food safety manager    35
Inadequate and inaccessible handwashing facilities                           35
Improper thawing methods                                                     30
Unclean hands or improper use of gloves                                      27
Improper cooling methods                                                     25
Unclean nonfood contact surfaces                                             21
Improper food storage       

<br/><br/><br/>

---

<br/><br/><br/>

# 6: Further analysis and pivots

In [345]:
#YOUR CODE HERE
twenty_score = ins_named[ins_named['score'] >= 0].sort_values(by=('score'), ascending=False)
ins_score_pivot = pd.pivot_table(twenty_score,index='name',values='score',aggfunc=[np.mean,np.median,min,max,np.std])
ins_score_pivot

#YOUR EXPLANATION HERE : Here I have added min,max,median, mean and standard deviation scores of each ins type. 
#This can help us evaluate how accuarte the avg.score is. And help represent the real picture.

,mean,median,min,max,std
,score,score,score,score,score
name,,,,,
"#1 VERJUS CAVE, #2 VERJUS",100.000000,100.0,100,100,NaN
111 Minna Gallery,95.333333,94.0,92,100,4.163332
12 Tribes Kosher Foods,98.000000,98.0,98,98,NaN
1428 Haight,77.000000,77.0,77,77,0.000000
15 Romolo,88.000000,88.0,88,88,NaN
...,...,...,...,...,...
Zpizza,93.333333,94.0,92,94,1.154701
Zynga,93.333333,94.0,92,94,1.154701


In [ ]:
#THIS CELL AND ANY CELLS ADDED BELOW WILL NOT BE GRADED



In our analysis of the business data, we found that there are some errors with the ZIP codes. As a result, we made the records with ZIP codes outside of San Francisco have a `postal5` value of `None` and shortened 9-digit zip codes to 5-digit ones. In practice, we could take the time to look up the restaurant address online and fix some of the zip code issues.

In our analysis of the inspection data, we investigated the relationship between the year and type of inspection, and we figured out that only `Routine - Unscheduled` inspections have inspection scores.
Finally, we joined the business and inspection data to identify restaurants with the worst ratings and the lowest median scores. 